__Subida de datos a [doccano](https://doccano.github.io/doccano/)__

__Paso 1:__
* Instalar la libreria: ```!pip install doccano```
* A continuación, iniciar el servidor:

```
# Initialize database. First time only.
doccano init
# Create a super user. First time only.
doccano createuser --username admin --password pass
# Start a web server.
doccano webserver --port 8000
```

* En otro terminal, ejecutar el siguiente comando:

```
doccano task
```

De cara a subir el dataset, seguir [los siguientes pasos](https://doccano.github.io/doccano/tutorial/).

In [15]:
# -- Convertimos la salida del modelo de OpenAI en el formato de doccano
import pandas as pd
import re
import os
output_df = pd.read_json("/Users/alberto/Personal Projects/hackathon_podcast/data/NER/json_entity/json_files_alberto/output_4.json")
output_df['fe_entities'] = output_df.apply(lambda x: [(x_aux['class'], x_aux['text'].lower(), re.search(x_aux['text'].lower(), x['text'].lower())) for x_aux in x['entities']\
                                                    if x_aux['text'] != "" and x_aux['text'] != None], axis=1)
output_df['fe_entities_start_end'] = output_df['fe_entities'].apply(lambda x: [[x_aux[-1].start(), x_aux[-1].end(), x_aux[0].upper()] for x_aux in x if x_aux[-1] != None])

__Doccano format__:

```
{"text": "EU rejects German call to boycott British lamb.", "label": [[0, 2, "ORG"]]}
{"text": "Peter Blackburn", "label": [[0, 15, "PERSON"]]}
{"text": "President Obama", "label": [[10, 15, "PERSON"]]}
```

In [16]:
output_df['fe_final_dict'] = output_df.apply(lambda x: {"text": x["text"], "label": x["fe_entities_start_end"]}, axis=1)
output_df

,text,entities,fe_entities,fe_entities_start_end,fe_final_dict
0,"Marruecos y Españita, ¿qué pasa aquí?\n","[{'class': 'places', 'text': 'Marruecos'}, {'c...","[(places, marruecos, <re.Match object; span=(0...","[[0, 9, PLACES]]","{'text': 'Marruecos y Españita, ¿qué pasa aquí..."
1,Vamos a contar la historia desde el principio.\n,"[{'class': 'topics', 'text': 'historia'}]","[(topics, historia, <re.Match object; span=(18...","[[18, 26, TOPICS]]",{'text': 'Vamos a contar la historia desde el ...
2,Ahí empezó el primer fuerte español en esa épo...,"[{'class': 'places', 'text': 'Ahí'}, {'class':...","[(places, ahí, <re.Match object; span=(0, 3), ...","[[0, 3, PLACES], [43, 48, DATES]]",{'text': 'Ahí empezó el primer fuerte español ...
3,La historia del Sájar Occidental comienza en 1...,"[{'class': 'places', 'text': 'Sájar Occidental...","[(places, sájar occidental, <re.Match object; ...","[[16, 32, PLACES], [45, 49, DATES]]",{'text': 'La historia del Sájar Occidental com...
4,Ahí nos estábamos dividiendo África como si fu...,"[{'class': 'places', 'text': 'África'}]","[(places, áfrica, <re.Match object; span=(29, ...","[[29, 35, PLACES]]",{'text': 'Ahí nos estábamos dividiendo África ...
...,...,...,...,...,...
95,"Y por eso yo creo que Argelia, aunque no lo di...","[{'class': 'people', 'text': 'Argelia'}, {'cla...","[(people, argelia, <re.Match object; span=(22,...","[[22, 29, PEOPLE], [133, 144, TOPICS], [105, 1...","{'text': 'Y por eso yo creo que Argelia, aunqu..."
96,"Ya a ver si van a ser ciudadanos de segunda, s...","[{'class': 'places', 'text': 'Marroquí'}]","[(places, marroquí, <re.Match object; span=(70...","[[70, 78, PLACES]]",{'text': 'Ya a ver si van a ser ciudadanos de ...
97,"Como los palestinos en el real, ¿no?\n","[{'class': 'topics', 'text': 'palestinos'}]","[(topics, palestinos, <re.Match object; span=(...","[[9, 19, TOPICS]]","{'text': 'Como los palestinos en el real, ¿no?..."
98,"Veremos, ya son magnánimos los marroquíes y lo...","[{'class': 'people', 'text': 'marroquíes'}, {'...","[(people, marroquíes, <re.Match object; span=(...","[[31, 41, PEOPLE], [110, 119, TOPICS], [121, 1...","{'text': 'Veremos, ya son magnánimos los marro..."


__Por último, exportamos el diccionario a .jsonl__

In [14]:
# Abre un archivo de salida en modo de escritura
import json
with open('/Users/alberto/Desktop/output.jsonl', 'w') as f:
    # Itera a través de las filas del DataFrame
    for _, row in output_df.iterrows():
        # Convierte el diccionario en una cadena JSON
        json_str = json.dumps(row['fe_final_dict'])
        # Escribe la cadena JSON en el archivo de salida, seguida de un salto de línea
        f.write(json_str + '\n')